In [2]:
from transformers import GPT2Model
from peft import LoraConfig, get_peft_model
import torch.nn as nn
import torch

model = GPT2Model.from_pretrained("gpt2")

In [3]:
model

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [4]:
total_params  = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params

124439808

In [5]:
class TiedTextHead(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, tied_weights=None):
        super().__init__()
        self.shared_mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU()
        )
        # output is twice vocab size
        # first half is for next token prediction: x_{t+1}
        # second half is for previous token prediction: x_{t+k-1}
        self.output_layer = nn.Linear(hidden_size, vocab_size * 2)

    def forward(self, f, b):
        combined = torch.cat([f, b], dim=-1)
        shared_output = self.shared_mlp(combined)
        logits = self.output_layer(shared_output)
        return logits

lora_config = LoraConfig(
            r=16, 
            lora_alpha=32, 
            lora_dropout=0.05, 
            # target_modules=["q_proj", "v_proj"],  # apply lora to attention layers
            bias="none",
            task_type="CAUSAL_LM",
        )

# create separate forward and backward lora adapters
model.add_adapter(lora_config, adapter_name="forward_encoder")
model.add_adapter(lora_config, adapter_name="backward_encoder")

# add tied text head for next and previous token predictions
text_head = TiedTextHead(
                    input_dim=model.config.hidden_size * 2,
                    hidden_size=512, # TODO; allow this to be configurable
                    vocab_size=50257,
                    # tied_weights=self.model.transformer.wte.weight  # use input embeddings' weights
                )


/Users/jaydenteoh/Library/Python/3.9/lib/python/site-packages/peft/tuners/lora/layer.py:1119: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [6]:
total_params  = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params

589824

In [1]:
from transformers import AutoModel, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from utils.training_utils import accuracy
from torch.nn import functional as F
import torch.nn as nn
import torch

class TiedTextHead(nn.Module):
    def __init__(self, input_dim, hidden_size, vocab_size, tied_weights=None):
        super().__init__()
        self.shared_mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_size),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.LeakyReLU()
        )
        # output is twice vocab size
        # first half is for next token prediction: x_{t+1}
        # second half is for previous token prediction: x_{t+k-1}
        self.output_layer = nn.Linear(hidden_size, vocab_size * 2)

    def forward(self, f, b):
        combined = torch.cat([f, b], dim=-1)
        shared_output = self.shared_mlp(combined)
        logits = self.output_layer(shared_output)
        return logits


class BeliefStateTransformer(nn.Module):
    def __init__(self, args):
        super().__init__()

        if args.load_in_4bit:
            quantization_config = BitsAndBytesConfig(
                                    load_in_4bit=True,
                                    bnb_4bit_compute_dtype=args.ptdtype
                                )
        else:
            # Assume default or no quantization
            quantization_config = None

        if args.use_flash:
            attn_implementation = "flash_attention_2"
        else:
            attn_implementation = None
        
        # TODO: support more pretrained models
        # import gpt2 with no specific head on top
        self.model = AutoModel.from_pretrained(
            args.model, 
            attn_implementation=attn_implementation,
            torch_dtype=args.ptdtype,
            quantization_config=quantization_config
        )

        # lora adapter config
        lora_config = LoraConfig(
            r=args.lora_r, 
            lora_alpha=args.lora_alpha, 
            lora_dropout=args.lora_dropout, 
            # target_modules=["q_proj", "v_proj"],  # apply lora to attention layers
            bias="none",
            task_type="CAUSAL_LM",
            fan_in_fan_out=True
        )

        # create separate forward and backward lora adapters
        self.model.add_adapter(lora_config, adapter_name="forward_encoder")
        self.model.add_adapter(lora_config, adapter_name="backward_encoder")

        # enable gradient checkpointing to save memory during training
        self.model.gradient_checkpointing_enable()

        # add tied text head for next and previous token predictions
        self.text_head = TiedTextHead(
                            input_dim=self.model.config.hidden_size * 2,
                            hidden_size=512, # TODO; allow this to be configurable
                            vocab_size=args.vocab_size,
                            # tied_weights=self.model.transformer.wte.weight  # use input embeddings' weights
                        )

    def forward(self, f, b):
        # forward encoding
        self.model.set_adapter("forward_encoder")
        forward_states = self.model(f).last_hidden_state  # get forward states

        # backward encoding
        self.model.set_adapter("backward_encoder")
        backward_input = torch.flip(b, dims=[1])  # reverse the input sequence
        backward_states = self.model(backward_input).last_hidden_state
        backward_states = torch.flip(backward_states, dims=[1])  # flip the backward states back

        # Text head for next and previous token predictions
        next_logits, prev_logits = self.text_head(forward_states, backward_states)
        # acc, token_acc = accuracy(logits, targets)
        # accs = {"acc": acc, "token_acc": token_acc}
        return next_logits, prev_logits
    
    def belief_state_objective(self, all_f, all_b, x):
        """
        Compute the belief state objective as described in the BST paper.
        """
        bs, T = x.shape
        forward_states = all_f
        backward_states = all_b.flip(1)
        # generate all valid combinations of forward and backward indices
        ft = torch.arange(T, dtype=torch.int32)  # forward indices
        bt = torch.arange(T, dtype=torch.int32)  # backward indices
        combinations = torch.cartesian_prod(ft, bt) 
        combinations = combinations[(combinations[:, 1] - combinations[:, 0] >= 2)]  # filter valid pairs
        fb_pairs = combinations[combinations[:, 1] < T]  # ensure backward indices are within range

        # extract valid indices
        f_idxs, b_idxs = fb_pairs[:, 0], fb_pairs[:, 1]
        nt_idxs = (combinations[:, 0] + 1)  # indices for next token labels

        # gather forward and backward features
        f = forward_states[:, f_idxs]
        b = backward_states[:, b_idxs]

        # prepare labels
        single_labels_f = x[:, nt_idxs].unsqueeze(2)  # labels for next-token prediction
        single_labels_b = x[:, b_idxs].unsqueeze(2)   # labels for prev-token prediction
        single_labels = torch.cat((single_labels_f, single_labels_b), dim=2)

        # compute logits from the text head
        logits = self.text_head(f, b)  # combine forward and backward states
        fb_numpairs = fb_pairs.shape[0]  # no of valid forward-backward pairs

        # reshape logits and labels for loss computation
        logits = logits.reshape((bs, fb_numpairs, 2, -1))  # split into next and previous logits
        logits = logits.reshape((bs * fb_numpairs * 2, -1))  # flatten for CEL
        single_labels = single_labels.reshape((bs * fb_numpairs * 2))  # flatten labels

        # compute the loss independently for next and previous token predictions
        # this also sums the negative log likelihood for 
        # all next and previous token predictions together, aligning with the paper
        loss = nn.CrossEntropyLoss()(logits, single_labels)
        return loss
    
    def update(self, x, optimizer, scaler, targets=None):
        """
        Efficient training for the BST model.
        """
        # Compute forward states
        self.model.set_adapter("forward_encoder")
        forward_states = self.model(x).last_hidden_state

        # Compute backward states
        self.model.set_adapter("backward_encoder")
        backward_input = torch.flip(x, dims=[1])
        backward_states = self.model(backward_input).last_hidden_state

        # Compute the combined loss
        loss = self.belief_state_objective(forward_states, backward_states, x)
        self.model.set_adapter(["forward_encoder", "backward_encoder"]) # IMPORTANT else gradient will only exist for backward_encoder

        return loss
    
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=1):
        """
        Generate text autoregressively using the forward model.

        Args:
            idx (torch.Tensor): Input tensor containing the tokenized prefix (batch_size, sequence_length).
            max_length (int): Maximum length of the generated sequence.
            temperature (float): Sampling temperature.
            top_k (int): Top-k sampling.

        Returns:
            torch.Tensor: Tokenized output sequence.
        """
        bsz, prefix_len = idx.shape
        device = idx.device

        # Step 1: Precompute backward latent state for empty suffix B(∅)
        empty_suffix = torch.zeros((bsz, 1), dtype=idx.dtype, device=device)
        self.model.set_adapter("backward_encoder")
        backward_state = self.model(empty_suffix).last_hidden_state[:, 0:1, :]  # Fixed backward state

        # Step 2: Initialize the generated sequence with the prefix
        out = idx.clone()

        for i in range(max_new_tokens):
            # Step 3: Compute forward latent state for current prefix F(x1:t)
            self.model.set_adapter("forward_encoder")
            forward_states = self.model(out).last_hidden_state

            # Step 4: Compute logits for the next token prediction
            logits = self.text_head(forward_states[:, -1:, :], backward_state)

            next_logits = logits[:, 0, :self.text_head.output_layer.out_features // 2]

            # Step 5: Apply temperature scaling
            next_logits = next_logits / temperature

            # Step 6: Apply top-k sampling if specified
            if top_k is not None:
                v, _ = torch.topk(next_logits, min(top_k, next_logits.size(-1)))
                next_logits[next_logits < v[:, [-1]]] = -float('Inf')

            # Step 7: Convert logits to probabilities and sample
            probs = F.softmax(next_logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)

            # Step 8: Append sampled index to the running sequence
            out = torch.cat((out, idx_next), dim=1)

        return out

    
    # def _create_optimizer(self, lr, weight_decay):
    #     return torch.optim.AdamW(
    #         [
    #             {"params": self.model.get_adapter("forward_encoder").parameters(), "lr": lr},
    #             {"params": self.model.get_adapter("backward_encoder").parameters(), "lr": lr},
    #             {"params": self.text_head.parameters(), "lr": lr}
    #         ],
    #         lr=lr,
    #         weight_decay=weight_decay
    #     )



In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, BitsAndBytesConfig
from peft import LoraConfig

# Assuming the BeliefStateTransformer class is already defined as provided earlier
# Define dummy arguments for the model
class Args:
    model = "bert-base-uncased"  # Example model
    load_in_4bit = False
    use_flash = False
    ptdtype = torch.float32
    lora_r = 4
    lora_alpha = 32
    lora_dropout = 0.1
    vocab_size = 30522  # Example vocab size

args = Args()

# Instantiate the model
model = BeliefStateTransformer(args)
model.train()

# Create dummy input data
batch_size = 2
seq_length = 10
dummy_x = torch.randint(0, args.vocab_size, (batch_size, seq_length))

# Define an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# Define a GradScaler for mixed precision (optional)
scaler = torch.cuda.amp.GradScaler(enabled=False)  # Disable for simplicity here

# Run one forward and backward pass
with torch.set_grad_enabled(True):
    model.model.set_adapter("forward_encoder")
    forward_states = model.model(dummy_x).last_hidden_state
    _f = forward_states.detach()
    _f.requires_grad = True

    # Compute backward states
    model.model.set_adapter("backward_encoder")
    backward_input = torch.flip(dummy_x, dims=[1])
    backward_states = model.model(backward_input).last_hidden_state
    _b = backward_states.detach()
    _b.requires_grad = True

    # Compute the combined loss
    loss = model.belief_state_objective(_f, _b, dummy_x)

    # compute text head gradients over all prefix/suffix pairs.
    scaled_loss = scaler.scale(loss)
    print("Loss:", scaled_loss.item())

scaled_loss.backward()
model.model.set_adapter("forward_encoder")
forward_states.backward(_f.grad)

model.model.set_adapter("backward_encoder")
backward_states.backward(_b.grad)

# model.model.set_adapter(["forward_encoder", "backward_encoder"])

# Check if gradients exist for specific parameters
print("\nChecking gradients for model parameters...")
for name, param in model.named_parameters():
    if param.grad is not None:
        print(f"{name}: {param.grad.norm().item()}")
    else:
        print(f"{name} has no gradient")

print("\nParameter norms before optimizer step:")
param_norms_before = {}
for name, param in model.named_parameters():
    if "forward_encoder" in name or "backward_encoder" in name:
        param_norms_before[name] = param.norm().item()
        print(f"{name}: {param.norm().item()}")

scaler.step(optimizer)
scaler.update()
optimizer.zero_grad(set_to_none=True)

print("\nParameter norms after optimizer step:")
for name, param in model.named_parameters():
    if name in param_norms_before:
        print(f"{name}: {param.norm().item()}, Change: {param.norm().item() - param_norms_before[name]}")


/Users/jaydenteoh/Library/Python/3.9/lib/python/site-packages/peft/tuners/lora/layer.py:1110: UserWarning: fan_in_fan_out is set to True but the target module is `torch.nn.Linear`. Setting fan_in_fan_out to False.
  warnings.warn(


Loss: 10.324504852294922

Parameter norms before optimizer step:
model.encoder.layer.0.attention.self.query.lora_A.forward_encoder.weight: 1.155055046081543
model.encoder.layer.0.attention.self.query.lora_A.backward_encoder.weight: 1.1544753313064575
model.encoder.layer.0.attention.self.query.lora_B.forward_encoder.weight: 0.0
model.encoder.layer.0.attention.self.query.lora_B.backward_encoder.weight: 0.0
model.encoder.layer.0.attention.self.value.lora_A.forward_encoder.weight: 1.156412124633789
model.encoder.layer.0.attention.self.value.lora_A.backward_encoder.weight: 1.1613956689834595
model.encoder.layer.0.attention.self.value.lora_B.forward_encoder.weight: 0.0
model.encoder.layer.0.attention.self.value.lora_B.backward_encoder.weight: 0.0
model.encoder.layer.1.attention.self.query.lora_A.forward_encoder.weight: 1.1700341701507568
model.encoder.layer.1.attention.self.query.lora_A.backward_encoder.weight: 1.1603801250457764
model.encoder.layer.1.attention.self.query.lora_B.forward_enco

In [19]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 142142324
Trainable parameters: 32660084


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

input_text = "This is a test."
input_ids = tokenizer(input_text, add_special_tokens=True)["input_ids"]
eos_token_id = tokenizer.eos_token_id
input_with_eos = input_ids + [eos_token_id]
print(tokenizer.decode(input_with_eos))

This is a test.<|endoftext|>
